In [187]:
import uproot
import pandas as pd
pd.set_option('display.max_colwidth', None)

import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use(hep.style.CMS)

In [188]:
with uproot.open("/home/users/aaarora/nvme/output/run2/sig.root") as f:
    df = f.get("Events").arrays([col for col in f.get("Events").keys() if "VBS" in col] + ["GenDeltaR1", "GenDeltaR2", "weight"], library="pd")

In [189]:
df[["GenDeltaR1", "GenDeltaR2", "TrueVBSJet1idx", "TrueVBSJet2idx"]]

,GenDeltaR1,GenDeltaR2,TrueVBSJet1idx,TrueVBSJet2idx
0,"[3.196406841278076, 3.3528096675872803, 4.285130023956299, 7.283416271209717, 0.035894691944122314, 7.353371620178223]","[4.329334259033203, 4.03207540512085, 3.66672945022583, 0.11388741433620453, 7.331027984619141, 0.6316060423851013]",4,3
1,"[3.827960729598999, 3.0572614669799805, 2.4224588871002197, 6.530063629150391, 0.04682590067386627, 0.7374122142791748, 3.446068286895752]","[4.201382160186768, 3.415804624557495, 4.590798377990723, 0.0806640237569809, 6.428288459777832, 7.189395904541016, 8.39810562133789]",4,3
2,"[5.8410725593566895, 5.232428550720215, 3.885000467300415, 8.667612075805664, 3.234156370162964, 0.05914555490016937, 2.936343193054199, 7.302043914794922]","[3.2019567489624023, 3.5262255668640137, 5.147008419036865, 0.029510876163840294, 5.607551097869873, 8.674029350280762, 5.7997307777404785, 1.9873921871185303]",5,3
3,"[4.348550796508789, 3.79032564163208, 5.427908897399902, 2.7039690017700195, 3.16144061088562, 0.04869387671351433, 3.9722721576690674, 4.768989562988281, 7.350991249084473, 5.906729698181152]","[2.825322389602661, 2.5687875747680664, 0.14837650954723358, 2.6357474327087402, 2.5987236499786377, 5.256516933441162, 3.0986135005950928, 0.5355977416038513, 2.0901873111724854, 1.7975043058395386]",5,2
4,"[3.361203670501709, 5.378451824188232, 5.171508312225342, 0.05620168149471283, 7.748459815979004, 5.793050289154053, 1.8891253471374512, 7.798855781555176]","[4.695863723754883, 2.326647996902466, 2.610382080078125, 7.633671283721924, 0.07725372910499573, 2.009702205657959, 5.927121162414551, 2.184896469116211]",3,4
...,...,...,...,...
17426,"[3.325735569000244, 3.4245595932006836, 4.8617329597473145, 2.904978036880493, 0.047076113522052765, 7.178948402404785, 8.002872467041016]","[5.066919326782227, 3.9906299114227295, 3.974215507507324, 5.251730442047119, 7.289219379425049, 0.2815125286579132, 0.8498073816299438]",4,5
17427,"[3.8985323905944824, 4.748938083648682, 6.455061912536621, 3.967296600341797, 6.565883636474609, 7.736208915710449, 0.0435044951736927, 8.733137130737305, 7.398675918579102, 8.003519058227539]","[4.035931587219238, 4.086607456207275, 2.999109983444214, 3.8373634815216064, 3.161168336868286, 0.02041787840425968, 7.783359050750732, 2.44684100151062, 0.8510677218437195, 1.1720750331878662]",6,5
17428,"[4.096776962280273, 3.529310464859009, 3.699587106704712, 0.06285201758146286, 6.437409400939941, 3.9695045948028564, 2.40374755859375, 7.795388221740723, 2.0962228775024414]","[4.055007457733154, 3.6102006435394287, 2.8144922256469727, 6.447901725769043, 0.02611391805112362, 3.928471326828003, 6.625324249267578, 3.177288055419922, 5.510824203491211]",3,4
17429,"[6.34552526473999, 6.578086853027344, 6.162002086639404, 5.345398426055908, 8.69324779510498, 10.454438209533691, 0.19114965200424194]","[4.566864490509033, 4.421547889709473, 4.762887477874756, 5.513264179229736, 2.115767002105713, 0.2943200170993805, 10.419476509094238]",6,5


In [190]:
maxE_eff = df[((df.TrueVBSJet1pt == df.VBSjet1pt) & (df.TrueVBSJet2pt == df.VBSjet2pt)) | ((df.TrueVBSJet1pt == df.VBSjet2pt) & (df.TrueVBSJet2pt == df.VBSjet1pt))].shape[0] / df.shape[0]

In [191]:
maxeta_eff = df[((df.TrueVBSJet1pt == df.VBSjet1pt_eta) & (df.TrueVBSJet2pt == df.VBSjet2pt_eta)) | ((df.TrueVBSJet1pt == df.VBSjet2pt_eta) & (df.TrueVBSJet2pt == df.VBSjet1pt_eta))].shape[0] / df.shape[0]

In [192]:
print("MaxE eff: ", maxE_eff)

MaxE eff:  0.8070104985370891


In [193]:
print("MaxEtaJJ eff: ", maxeta_eff)

MaxEtaJJ eff:  0.7725316964029603
